In [1]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd
from core.analysis import access_db, get_senses, fb_to_local
from core import semcor_bert_pipeline
from core.metrics import *

<Figure size 432x288 with 0 Axes>

## Binary Logistic Regression

For the shared words, we find that homonymous and polysemous senses are able to be distinguished

In [2]:
shared_metrics = {}
shared_words = ['foot.n', 'plane.n', 'model.n', 'table.n', 'degree.n', 'right.n']
fmt_sense_12 = lambda w: [w + '.01', w + '.02']
shared_senses = [fmt_sense_12(w) for w in shared_words]
shared_senses[-1] = ['degree.n.01', 'academic_degree.n.01']
shared_senses[1] = ['airplane.n.01', 'plane.n.02']
for w_s in zip(shared_words, shared_senses):
    model_data = binary_logistic(w_s[0], w_s[1])
    weight_values, weight_indices = nonzero_weights(model_data['model'])
    f_scores, accuracies, wrong_indices = k_fold_cv(model_data['data'], model_data['transformed_labels'], k = )
    shared_metrics[w_s[0]] = {'senses': w_s[1], 'data': model_data, 'weights': weight_values,
                    'weight_indices': weight_indices, 'f1_kfold': f_scores, 'acc_kfold': accuracies,
                             'incorrect_indices': wrong_indices}
    

/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [3]:
shared_stats = []
incorrect = []
for k in shared_metrics:
    word_results = shared_metrics[k]
    weights = word_results['weights']
    incorrect.append(misclassified_sentences(word_results['data'], word_results['incorrect_indices']))
    shared_stats.append({'word_type': k, 'avg_f1': np.mean(word_results['f1_kfold']), 'avg_acc': np.mean(word_results['acc_kfold']),
    'pct_nonzero_weights': len(weights) / 768, 'max_wt': max(weights), 'min_wt': min(weights), 'mean_nonzero_wt': np.mean(weights), 'sd_nonzero_wt': np.std(weights)})
pd.DataFrame(shared_stats)

,word_type,avg_f1,avg_acc,pct_nonzero_weights,max_wt,min_wt,mean_nonzero_wt,sd_nonzero_wt
0,foot.n,1.000000,1.000000,0.031250,0.221638,-0.229722,0.028953,0.132456
1,plane.n,0.933333,0.980000,0.015625,0.491381,-0.265000,0.054483,0.223284
2,model.n,0.518095,0.885455,0.037760,0.274155,-0.291541,0.020775,0.136917
3,table.n,0.992000,0.993548,0.033854,0.475794,-0.253788,0.013552,0.151044
4,degree.n,1.000000,1.000000,0.018229,0.456742,-0.227727,0.001208,0.166292


Misclassified senses (Only binary classification, 3/16 senses for math plane were misclassified)

In [4]:
pd.set_option('display.max_colwidth', 500)
pd.concat(incorrect)

,true_label,sentences
0,plane.n.02,The plane of the action in the scene is not parallel with the plane of the film in the camera or on the screen .
0,model.n.01,His first model arrived at dusk .
1,model.n.01,"We devote a chapter to the binomial distribution not only because it is a mathematical model for an enormous variety of real life phenomena , but also because it has important properties that recur in many other probability models ."
2,model.n.01,His first model arrived at dusk .
3,model.n.01,His first model arrived at dusk .
4,model.n.01,The model quite plainly thought Michelangelo crazy ; only the instructions from his rabbi kept him from bolting .
5,model.n.01,"But at the end of the sitting , when Michelangelo showed him the quick , free drawings , with the mother roughed in , holding her son , the model grasped what Michelangelo was after , and promised to speak to his friends ."
0,table.n.01,The registration figures given in Table 2 must be interpreted with caution since the estimate for eligible electors were made without the benefit of a reliable census .


I thought this might be useful to see if weights at similar positions were used.

In [7]:
[(k, shared_metrics[k]['weight_indices']) for k in shared_words]

[('foot.n',
  array([ 13,  29,  40,  70,  89, 184, 191, 217, 231, 278, 287, 304, 308,
         332, 493, 518, 528, 532, 547, 637, 664, 693, 709, 730])),
 ('plane.n',
  array([105, 157, 254, 286, 308, 324, 411, 413, 448, 513, 539, 619])),
 ('model.n',
  array([ 18,  49,  68,  74,  79, 101, 114, 242, 254, 259, 282, 289, 340,
         364, 450, 473, 477, 509, 520, 523, 525, 527, 552, 574, 586, 597,
         618, 739, 749])),
 ('table.n',
  array([ 37,  67,  94, 136, 143, 188, 213, 228, 252, 286, 298, 317, 332,
         334, 363, 376, 393, 445, 481, 487, 552, 661, 666, 680, 705, 753])),
 ('degree.n',
  array([ 15,  22, 220, 262, 328, 350, 432, 523, 537, 541, 544, 620, 622,
         724]))]

## All Classes

In [14]:
def k_fold_cv(x, y, k = 5, labels = []):
    kf = KFold(n_splits = k, shuffle = True)
    f = []
    acc = []
    incorrect_indices = []
    confusion_matrices = []
    for train_index, test_index in kf.split(x):
        model = LogisticRegression(penalty = 'l1', multi_class = 'multinomial', solver = 'saga', max_iter = 5000)
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        test_pred = model.predict(X_test)
        #print(classification_report(y_test, test_pred))
        f.append(f1_score(y_test, test_pred, average = "weighted"))
        acc.append(accuracy_score(y_test, test_pred))
        print(accuracy_score(y_test, test_pred))
        print(y_test, test_pred)
        print(confusion_matrix(y_test, test_pred))
        confusion_matrices.append(confusion_matrix(y_test, test_pred))
        incorrect_indices +=[i[0] for i in np.argwhere(y_test != test_pred)]
    return f, acc, incorrect_indices, confusion_matrices


In [26]:
def logistic_cv(lemma, sel_senses = [], use_masc = True):
    name, pos = lemma.split(".")
    data = semcor_bert_pipeline.load_data(name, pos, 'semcor')
    embeddings = data['embeddings']
    sense_labels = data['sense_labels']
    strip_synset = lambda s: s.strip("Synset()").strip("'")
    target_senses = [strip_synset(i) for i in data['sense_names']]
    try:
        masc_data = semcor_bert_pipeline.load_data(name, pos, 'masc')
        embeddings += masc_data['embeddings']
        sense_labels += masc_data['sense_labels']
    except:
        pass
    le = LabelEncoder()
    le.fit(target_senses)
    x = np.asarray(embeddings)
    y = le.transform(sense_labels)
    if len(sel_senses):
        print(len(x))
        sense_indices = [i for i in range(len(sense_labels)) if sense_labels[i] in sel_senses]
        print(sense_indices)
        x = x[sense_indices]
        y = y[sense_indices]
        print(len(x))
    model = LogisticRegression(penalty = 'l1', solver = 'saga', max_iter = 10000)
    model.fit(x, y)
    #weight_values, weight_indices = nonzero_weights(model)
    f_scores, accuracies, wrong_indices, confusion_matrices = k_fold_cv(x, y, k = 5, labels = target_senses)
    return {'model': model, "data": x, "labels": sense_labels, "acc": accuracies, "f1": f_scores, 
            'incorrect_indices': wrong_indices, 'sentences': np.asarray(data['original_sentences']),
           'confusion_matrices': confusion_matrices}
    



In [12]:
db = access_db()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [16]:
model_results = logistic_cv('table.n', sel_senses = [fb_to_local(s) for s in get_senses(db, 'table_n')])

0.9375
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0] [0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0]
[[12  1]
 [ 1 18]]
0.967741935483871
[0 0 0 0 0 0 0 0 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0] [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0]
[[11  0  0]
 [ 0 19  0]
 [ 0  1  0]]
0.967741935483871
[0 0 0 0 0 0 0 0 0 0 0 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0] [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
[[15  0  0]
 [ 0 15  0]
 [ 0  1  0]]
1.0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1]
[[19  0]
 [ 0 12]]
0.9354838709677419
[0 0 0 0 0 0 0 0 0 0 2 2 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0] [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0]
[[13  0  0]
 [ 0 16  0]
 [ 0  2  0]]


In [17]:
model_results['confusion_matrices']

[array([[12,  1],
        [ 1, 18]]), array([[11,  0,  0],
        [ 0, 19,  0],
        [ 0,  1,  0]]), array([[15,  0,  0],
        [ 0, 15,  0],
        [ 0,  1,  0]]), array([[19,  0],
        [ 0, 12]]), array([[13,  0,  0],
        [ 0, 16,  0],
        [ 0,  2,  0]])]

In [27]:
shared_model_data = {}
shared_stats = []
incorrect = []
shared_words = ['foot.n', 'plane.n', 'model.n', 'table.n', 'degree.n', 'right.n']
for w in shared_words:
    print(w)
    print([fb_to_local(s) for s in get_senses(db, w.replace('.n', "_n"))])

    model_results = logistic_cv(w, sel_senses = [fb_to_local(s) for s in get_senses(db, w.replace('.n', "_n"))])
    shared_model_data[w] = model_results
    shared_stats.append({"word": w, 'avg_f1': np.mean(model_results['f1']),
                         'avg_acc': np.mean(model_results['acc'])})    
    #incorrect.append(misclassified_sentences(model_results, model_results['incorrect_indices']))

foot.n
['foot.n.01', 'foot.n.02', 'foot.n.03']
162
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161]
158
1.0
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[17  0]
 [ 0 15]]
1.0
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 3 1] [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


1.0
[1 1 1 1 0 0 0 0 0] [1 1 1 1 0 0 0 0 0]
[[5 0]
 [0 4]]
model.n
['model.n.01', 'model.n.02', 'model.n.03']
72
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
65
1.0
[3 3 1 1 1 2 1 1 1 1 1 1 3] [3 3 1 1 1 2 1 1 1 1 1 1 3]
[[9 0 0]
 [0 1 0]
 [0 0 3]]
0.6153846153846154
[3 3 1 1 1 2 2 2 1 1 1 3 1] [1 3 1 1 1 3 2 1 1 1 1 2 2]
[[6 1 0]
 [1 1 1]
 [1 1 1]]
1.0
[3 1 1 1 2 2 2 1 1 1 1 1 1] [3 1 1 1 2 2 2 1 1 1 1 1 1]
[[9 0 0]
 [0 3 0]
 [0 0 1]]
0.7692307692307693
[1 1 1 1 1 1 2 2 1 1 1 1 3] [1 1 1 1 1 1 3 3 1 1 1 1 1]
[[10  0  0]
 [ 0  0  2]
 [ 1  0  0]]
0.8461538461538461
[3 1 1 1 1 2 2 1 1 1 3 3 1] [3 1 1 1 1 2 2 1 1 1 2 2 1]
[[8 0 0]
 [0 2 0]
 [0 2 1]]
table.n
['table.n.01', 'table.n.02', 'table.n.03']
156
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22

In [81]:
pd.DataFrame(shared_stats)

,word,avg_f1,avg_acc
0,foot.n,0.974961,0.981629
1,plane.n,0.922500,0.940000
2,model.n,0.745612,0.765714
3,table.n,0.952730,0.961694
4,degree.n,0.682346,0.700000
5,right.n,0.887547,0.901282


In [79]:
shared_words

['foot.n', 'plane.n', 'model.n', 'table.n', 'degree.n']